In [61]:
from sklearn.datasets import fetch_openml

data = fetch_openml(name="boston", version=1, as_frame=False)
X, y = data.data, data.target


In [62]:
import torch
import tensorflow as tf

In [63]:
import sys
sys.path.append(r'D:\Maths Y4\Project\TensorBNN')

# Now you can import TensorBNN or any module within it
from tensorBNN.layer import DenseLayer
from tensorBNN.network import network
from tensorBNN.activationFunctions import Relu
from tensorBNN.likelihood import GaussianLikelihood
from tensorBNN.metrics import SquaredError
from tensorBNN.metrics import PercentError

In [64]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [65]:
def give_me_data():
    import numpy as np
    import torch
    from torch import nn
    from sklearn.model_selection import train_test_split
    from tqdm import tqdm
    def unnormalised_ground_truth(x):
        """return the 'ground truth' output for a specific input location x"""
        return 15*np.cos(0.1/(0.05*(x+1)+0.02)) + 3*np.exp(0.5*np.sin(((50*(x+1))**0.9-3)/5))/((50*(x+1)+0.01)**(-0.5)) -12.5*(x+1)**2

    def normalised_ground_truth(xs):
        ys = unnormalised_ground_truth(xs)
        m = ys.mean()
        s = ys.std()
        return (ys - m) / s

    xs = np.linspace(-1, 1, 1000)
    ys = normalised_ground_truth(xs)

    def generate_dataset(size=50, noise=0.15, split=0.3):
        xs = np.linspace(-1, 1, 1000)
        ys = normalised_ground_truth(xs)
        samp_ind = np.random.randint(0, 999, size)
        x_samps = xs[samp_ind]
        y_samps = ys[samp_ind] + np.random.normal(0, noise, size)
        return train_test_split(x_samps, y_samps, test_size=split, random_state=13)

    x_tr, x_te, y_tr, y_te = generate_dataset()
    # x_tr = torch.FloatTensor(x_tr).unsqueeze(-1)
    # y_tr = torch.FloatTensor(y_tr).unsqueeze(-1)
    # x_te = torch.FloatTensor(x_te).unsqueeze(-1)
    # y_te = torch.FloatTensor(y_te).unsqueeze(-1) 

    # Convert NumPy arrays to TensorFlow tensors
    x_tr_tf = tf.convert_to_tensor(x_tr, dtype=tf.float32)
    y_tr_tf = tf.convert_to_tensor(y_tr, dtype=tf.float32)
    x_te_tf = tf.convert_to_tensor(x_te, dtype=tf.float32)
    y_te_tf = tf.convert_to_tensor(y_te, dtype=tf.float32)
   
    return x_tr_tf, y_tr_tf, x_te_tf, y_te_tf

In [66]:
x_tr,y_tr,x_te,y_te = give_me_data()
y_tr.dtype

tf.float32

In [89]:
neuralNet = network(dtype=tf.float32, inputDims=1, trainX=x_tr,trainY= y_tr,validateX= x_te,validateY= y_te)

In [90]:
neuralNet.add(DenseLayer(1, 2, dtype=tf.float32))
neuralNet.add(Relu())
neuralNet.add(DenseLayer(2, 1, dtype=tf.float32))

In [91]:
neuralNet.setupMCMC(stepSizeStart=1e-3, stepSizeMin=1e-4, stepSizeMax=1e-2,
                  stepSizeOptions=40, leapfrogStart=1000, leapfogMin=100,
                  leapFrogMax=10000, leapfrogIncrement=1, hyperStepSize=1e-2,
                  hyperLeapfrog=100, burnin=1000,
                  cores=4, averagingSteps=10, a=4, delta=0.1, strikes=5,
                  randomSteps=10, dualAveraging=False)

In [92]:
neuralNet.burnin

1000

In [93]:
# Declare Gaussian Likelihood with sd of 0.1
likelihood =  GaussianLikelihood(sd = 0.1)
metricList = [ #Declare metrics
    SquaredError(mean = 0, sd = 1, scaleExp = False),
    PercentError(mean = 10, sd = 2, scaleExp = True)]

In [94]:
neuralNet.train(
        epochs= 1000, # epochs to train for
        samplingStep= 5, # increment between network saves
        likelihood=likelihood,
        metricList = metricList,
        folderName = "Regression", 
        # Name of folder for saved networks
        networksPerFile=50)
        # Number of networks saved per file


iter: 1
step size 0.001
hyper step size 0.80580854
leapfrog 1000
Main acceptance 1.1883829e-05
Hyper acceptance 0.0
training squared error  3.99994 validation squared error  2.27408
training percent error 67739.609 validation percent error 8451.083
Loss Standard Deviation:  0.1
Time elapsed: 9.896152973175049

iter: 2
step size 0.001
hyper step size 0.65679485
leapfrog 1000
Main acceptance 3.0238735e-19
Hyper acceptance 0.0
training squared error  3.99994 validation squared error  2.27408
training percent error 67739.609 validation percent error 8451.083
SJD: 0.0
Loss Standard Deviation:  0.1
Time elapsed: 9.428685426712036

iter: 3
step size 0.001
hyper step size 0.5207874
leapfrog 1000
Main acceptance 9.474614e-06
Hyper acceptance 0.0
training squared error  3.99994 validation squared error  2.27408
training percent error 67739.609 validation percent error 8451.083
SJD: 0.0
Loss Standard Deviation:  0.1
Time elapsed: 0.016996145248413086

iter: 4
step size 0.001
hyper step size 0.40

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
from TensorBNN.predictor import predictor 

network = predictor(filePath,
                    dtype = tf.float32, 
                    # data type used by network
                    customLayerDict={"dense2": Dense2},
                    # A dense layer with a different 
                    # hyperprior
                    likelihood = likelihood)
                    # The likelihood function is required to  
                    # calculate the probabilities for 
                    # re-weighting

